# 导入必要库

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [2]:
from keras.preprocessing import sequence 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 

from keras.optimizers import adam_v2
from keras.models import load_model 
from keras.callbacks import ModelCheckpoint

# 预览数据

In [3]:
# 一期数据
df = pd.read_csv("10-15一期实验数据/10-15-001.csv", header=None, usecols=[0,1,2,3,4,5])
df.head()

,0,1,2,3,4,5
0,9856,-13888,-2292,-904,-136,-170
1,9764,-13860,-2212,-602,-403,-35
2,9696,-14000,-2020,-644,-184,18
3,10020,-13732,-2276,-575,-418,-42
4,9844,-13880,-2096,-733,-397,24


In [4]:
# 二期数据
df = pd.read_csv("11.28实验2/back001.csv", header=None, usecols=[0,1,2,3,4,5])
df.head()

,0,1,2,3,4,5
0,-4180,-16092,-276,-690,-999,23
1,-4016,-16112,284,-789,-628,153
2,-4228,-15780,56,-917,-571,17
3,-3976,-16080,-276,-691,-583,35
4,-4400,-15960,-156,-586,-160,63


# 处理数据长度

In [3]:
# 一期数据
path_1 = "10-15一期实验数据/"
# 二期数据
path_2 = "11.28实验2/"
sequences = list()
# 正样本数据
for i in range(1, 39):
    if i < 10:
        filePath = path_1 + "10-15-00" + str(i) + ".csv"
    else:
        filePath = path_1 + "10-15-0" + str(i) + ".csv"
    print(filePath)
    df = pd.read_csv(filePath, header=None, usecols=[0,1,2,3,4,5]).dropna(axis=0,how='any')
    values = df.values
    sequences.append(values)
for i in range(1, 8):
    filePath = path_2 + "back00" + str(i) + ".csv"
    df = pd.read_csv(filePath, header=None, usecols=[0,1,2,3,4,5]).dropna(axis=0,how='any')
    print(filePath)
    values = df.values
    sequences.append(values)
for i in range(1, 41):
    if i < 10:
        filePath = path_2 + "left00" + str(i) + ".csv"
    else:
        filePath = path_2 + "left0" + str(i) + ".csv"
    print(filePath)
    df = pd.read_csv(filePath, header=None, usecols=[0,1,2,3,4,5]).dropna(axis=0,how='any')
    values = df.values
    sequences.append(values)
    
# 负样本数据
for i in range(1, 20):
    filePath = path_1 + "0-" + str(i) + ".csv"
    print(filePath)
    df = pd.read_csv(filePath, header=None, usecols=[0,1,2,3,4,5]).dropna(axis=0,how='any')
    values = df.values
    sequences.append(values)
for i in range(1, 4):
    filePath = path_2 + "jump00" + str(i) + ".csv"
    print(filePath)
    df = pd.read_csv(filePath, header=None, usecols=[0,1,2,3,4,5]).dropna(axis=0,how='any')
    values = df.values
    sequences.append(values)

10-15一期实验数据/10-15-001.csv
10-15一期实验数据/10-15-002.csv
10-15一期实验数据/10-15-003.csv
10-15一期实验数据/10-15-004.csv
10-15一期实验数据/10-15-005.csv
10-15一期实验数据/10-15-006.csv
10-15一期实验数据/10-15-007.csv
10-15一期实验数据/10-15-008.csv
10-15一期实验数据/10-15-009.csv
10-15一期实验数据/10-15-010.csv
10-15一期实验数据/10-15-011.csv
10-15一期实验数据/10-15-012.csv
10-15一期实验数据/10-15-013.csv
10-15一期实验数据/10-15-014.csv
10-15一期实验数据/10-15-015.csv
10-15一期实验数据/10-15-016.csv
10-15一期实验数据/10-15-017.csv
10-15一期实验数据/10-15-018.csv
10-15一期实验数据/10-15-019.csv
10-15一期实验数据/10-15-020.csv
10-15一期实验数据/10-15-021.csv
10-15一期实验数据/10-15-022.csv
10-15一期实验数据/10-15-023.csv
10-15一期实验数据/10-15-024.csv
10-15一期实验数据/10-15-025.csv
10-15一期实验数据/10-15-026.csv
10-15一期实验数据/10-15-027.csv
10-15一期实验数据/10-15-028.csv
10-15一期实验数据/10-15-029.csv
10-15一期实验数据/10-15-030.csv
10-15一期实验数据/10-15-031.csv
10-15一期实验数据/10-15-032.csv
10-15一期实验数据/10-15-033.csv
10-15一期实验数据/10-15-034.csv
10-15一期实验数据/10-15-035.csv
10-15一期实验数据/10-15-036.csv
10-15一期实验数据/10-15-037.csv
10-15一期实验数据/10-15-038.csv
11.28实验2/bac

In [12]:
targets = pd.read_csv("11.28实验2/target.csv", header=None).values

**由于时间序列数据长度不一，我们不能直接在这个数据集上建立模型。先找出最小、最大和平均长度：**

In [13]:
len_sequences = [] 
for one_seq in sequences: 
    len_sequences.append(len(one_seq)) 
pd.Series(len_sequences).describe()

count    107.000000
mean      29.373832
std        7.613944
min       13.000000
25%       26.000000
50%       29.000000
75%       31.500000
max       84.000000
dtype: float64

**大多数文件的长度在$20$到$40$之间。只有极少个文件的长度超过$60$。因此，取最小或最大长度没有多大意义。选择上四分位数作为序列的长度，进行数据处理**

In [14]:
#用最后一行的值填充序列到最大长度
to_pad = 84
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(6, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#将序列截断为长度 32
from keras.preprocessing import sequence
seq_len = 32
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [15]:
final_seq.shape

(107, 32, 6)

# 构建训练集和测试集

In [16]:
# 引入训练集、测试集划分函数
from sklearn.model_selection import train_test_split

# 将数据集按照 80% 和 20% 的比例随机划分为训练集和测试集
train_data, test_data, train_target, test_target = train_test_split(final_seq, targets)
test_target.shape

(27, 1)

In [44]:
train_data = np.array(train_data)
test_data = np.array(test_data)

train_target = np.array(train_target)
#train_target = (train_target+1)/2

test_target = np.array(test_target)
#test_target = (test_target+1)/2

# 构建 LSTM

In [56]:
from keras.layers import Dropout
model = Sequential() 
model.add(LSTM(256, input_shape=(seq_len, 6), return_sequences=True)) 
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(16, activation='ReLU'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 32, 256)           269312    
                                                                 
 lstm_9 (LSTM)               (None, 32, 128)           197120    
                                                                 
 lstm_10 (LSTM)              (None, 32, 64)            49408     
                                                                 
 lstm_11 (LSTM)              (None, 32)                12416     
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [58]:
from keras.callbacks import TensorBoard
adam = adam_v2.Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train_data, train_target, epochs=100, batch_size=128, callbacks=[TensorBoard(log_dir='./logs')])

D:\Python-Anaconda3-64bit\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 0.6997 - accuracy: 0.4250
Epoch 2/100
1/1 [==============================] - 0s 201ms/step - loss: 0.6273 - accuracy: 0.7500
Epoch 3/100
1/1 [==============================] - 0s 189ms/step - loss: 0.5422 - accuracy: 0.8625
Epoch 4/100
1/1 [==============================] - 0s 156ms/step - loss: 0.4865 - accuracy: 0.9000
Epoch 5/100
1/1 [==============================] - 0s 157ms/step - loss: 0.4466 - accuracy: 0.8750
Epoch 6/100
1/1 [==============================] - 0s 157ms/step - loss: 0.3678 - accuracy: 0.9000
Epoch 7/100
1/1 [==============================] - 0s 157ms/step - loss: 0.3169 - accuracy: 0.9125
Epoch 8/100
1/1 [==============================] - 0s 157ms/step - loss: 0.3012 - accuracy: 0.9375
Epoch 9/100
1/1 [==============================] - 0s 179ms/step - loss: 0.2866 - accuracy: 0.9250
Epoch 10/100
1/1 [==============================] - 0s 189ms/step - loss: 0.2757 - accuracy: 0.9250
Epoch 11/100

1/1 [==============================] - 0s 157ms/step - loss: 0.1657 - accuracy: 0.9500
Epoch 84/100
1/1 [==============================] - 0s 146ms/step - loss: 0.0954 - accuracy: 0.9625
Epoch 85/100
1/1 [==============================] - 0s 157ms/step - loss: 0.0852 - accuracy: 0.9750
Epoch 86/100
1/1 [==============================] - 0s 156ms/step - loss: 0.0990 - accuracy: 0.9750
Epoch 87/100
1/1 [==============================] - 0s 179ms/step - loss: 0.0845 - accuracy: 0.9875
Epoch 88/100
1/1 [==============================] - 0s 157ms/step - loss: 0.0708 - accuracy: 0.9875
Epoch 89/100
1/1 [==============================] - 0s 169ms/step - loss: 0.0668 - accuracy: 0.9750
Epoch 90/100
1/1 [==============================] - 0s 157ms/step - loss: 0.0861 - accuracy: 0.9750
Epoch 91/100
1/1 [==============================] - 0s 169ms/step - loss: 0.0805 - accuracy: 0.9625
Epoch 92/100
1/1 [==============================] - 0s 179ms/step - loss: 0.0638 - accuracy: 0.9750
Epoch 93/100


# 检测结果

In [70]:
#model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
#test_preds = model.predict_classes(test_data)
test_preds = model.predict(test_data)
for i in range(len(test_preds)):
    if test_preds[i] > 0.5:
        test_preds[i] = 1
    else:
        test_preds[i] = 0
print("模型准确率为：")
print(accuracy_score(test_target, test_preds)*100, "%")

模型准确率为：
100.0 %


In [3]:
from tensorboard import notebook
notebook.list()
notebook.start("--logdir logs --port 8122")

Known TensorBoard instances:
  - port 8123: logdir logs (started 0:14:05 ago; pid 19564)


# 保存模型

In [68]:
model.save("LSTM9629.h5")

In [69]:
from tensorflow.keras.models import load_model
model = load_model("LSTM9629.h5")
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 32, 256)           269312    
                                                                 
 lstm_9 (LSTM)               (None, 32, 128)           197120    
                                                                 
 lstm_10 (LSTM)              (None, 32, 64)            49408     
                                                                 
 lstm_11 (LSTM)              (None, 32)                12416     
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [43]:
predict_x = model.predict(test_data[0][None,:,:])
classes_x=np.argmax(predict_x,axis=1)
print(classes_x)
print(predict_x)
print(test_target[0])

[0]
[[2.6852252]]
[1]
